In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict

In [17]:
from dataloader import read_bci_data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [18]:
train_data, train_label, test_data, test_label = read_bci_data()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


## Implement EEGNet

In [15]:
class EEGNet(nn.Module):
    def __init__(self, in_channel = 2):
        super(EEGNet, self).__init__()
        layers = OrderedDict()
        layers['conv1'] = nn.Conv2d(in_channel,1,16)
        
    def forward(self):
        pass